In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

**Read in MTV artist files and merge for scrape (using these files to specify genre)**

In [77]:
mtv1 = pd.read_csv('data/10000-MTV-Music-Artists-page-1.csv')
mtv2 = pd.read_csv('data/10000-MTV-Music-Artists-page-2.csv')
mtv3 = pd.read_csv('data/10000-MTV-Music-Artists-page-3.csv')
mtv4 = pd.read_csv('data/10000-MTV-Music-Artists-page-4.csv')

In [78]:
mtv_artists = pd.concat([mtv1, mtv2, mtv3, mtv4], ignore_index=True)

In [79]:
mtv_artists.shape

(8344, 6)

In [80]:
mtv_artists.head()

,name,facebook,twitter,website,genre,mtv
0,Adele,http://www.facebook.com/9770929278,http://www.twitter.com/officialadele,NaN,Pop,http://www.mtv.com/artists/adele/biography
1,Joey + Rory,http://www.facebook.com/15044507815,http://www.twitter.com/joeyandrory,NaN,Country,http://www.cmt.com/artists/joey-rory/biography
2,Draaco Aventura,http://www.facebook.com/856796091053581,http://www.twitter.com/DraacoAventura,http://www.bandpage.com/draacoaventura,Pop Latino,http://www.mtv.com/artists/draaco-aventura/bio...
3,Justin Bieber,http://www.facebook.com/309570926875,http://www.twitter.com/justinbieber,http://www.justinbiebermusic.com,Pop,http://www.mtv.com/artists/justin-bieber/biogr...
4,Peer van Mladen,http://www.facebook.com/264487966,http://www.twitter.com/Predrag_Jugovic,http://pejaintergroup.eu/Peer_van_Mladen.html,House,http://www.mtv.com/artists/peer-van-mladen/bio...


**Drop unnecessary columns and fill n/a will NaN. Will skip over in scrape**

In [81]:
mtv_artists.drop(columns=['facebook', 'twitter', 'website', 'mtv'], inplace=True)

In [82]:
mtv_artists.fillna('NaN', inplace=True) #make sure name column is dtype object

**Select genres I want to focus on. For my purposes I prefer alternative/indie, so that's what I'm going with here.**

In [83]:
mtv_artists['genre'].value_counts().head(20)

NaN                     3226
Rock                     899
Country                  645
Hip-Hop/Rap              386
Pop                      385
Alternative              336
R&B/Soul                 217
Hip-Hop                  200
Latin                    146
Jazz                     133
World/Reggae              91
Pop/Rock                  82
Soul                      80
Indie Rock                80
Contemporary Country      80
Blues/Folk                79
Classic Rock              61
Electronic                61
Inspirational             59
Electronic/Dance          53
Name: genre, dtype: int64

In [114]:
set([genre for genre in mtv_artists['genre'] if 'alternative' in genre.lower()
    or 'indie' in genre.lower()])

{'Adult Alternative',
 'Alternative',
 'Alternative Country',
 'Alternative Folk',
 'Alternative Rap',
 'Indie Rock'}

**Ended up using mostly rock and alternative**

In [121]:
final_artists = mtv_artists[mtv_artists['genre'].isin(['Alternative', 'Indie Rock',
                                                       'Alternative Rock', 'Folk-Rock'])]

In [122]:
artist_lst = [name.strip() for name in final_artists.name]

In [123]:
len(artist_lst)

429

**Import spotipy and set credentials (remove these when making public on github)**

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

**First get artist information and URI (unique artist identifier needed to obtain albums)**

In [125]:
import time

def get_artist_info(artist_lst):
    artist_dict = []
    counter = 0
    for artist in artist_lst:
        counter += 1
        if counter % 200 == 0:
            time.sleep(3) #add 3 seconds to avoid being rate-limited by spotify
        try:
            name = artist #chosen artist
            result = sp.search(name) #search query
            artist_dict.append(result['tracks']['items'][0]['artists'])
            
        #Extract Artist's uri
            artist_uri = result['tracks']['items'][0]['artists'][0]['uri']
        except:
            pass
    return artist_dict

In [126]:
artist_dict = get_artist_info(artist_lst)

In [127]:
def get_artist_uris():
    uris = []
    for elem in artist_dict:
        for k, v in elem[0].items():
            if k =='uri':
                uris.append(v)
    return uris   

In [128]:
uris = get_artist_uris()

**Function for album names**

In [129]:
def get_album_names():
    album_names = []
    for uri in uris:
        sp_albums = sp.artist_albums(uri, album_type='album')
        for d in sp_albums['items']:
            for k, v in d.items():
                if k =='name':
                    album_names.append(v)
    return album_names

In [131]:
album_names = get_album_names()
album_names = list(set(album_names))

**Function for album uri**

In [133]:
def get_album_uris():
    album_uris = []
    for uri in uris:
        sp_albums = sp.artist_albums(uri, album_type='album')
        for d in sp_albums['items']:
            for k, v in d.items():
                if k =='uri':
                    album_uris.append(v)
    return album_uris

In [134]:
album_uris = get_album_uris()

**Get each song from album**

In [135]:
def album_songs(uri):
    album = uri 
    spotify_albums[album] = {}
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    #pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(album_uris[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [136]:
#help for code below:
#https://www.kaggle.com/pavansanagapati/spotify-music-api-data-extraction-part1
#https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37

In [137]:
spotify_albums = {}
album_count = 0
for i in album_uris: #each album
    album_songs(i)
    album_count+=1
album_count

4781

**Function for each song feature from each song of each album scraped**

In [138]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        try:
            features = sp.audio_features(track)

            
        #Append to relevant key-value
            spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
            spotify_albums[album]['danceability'].append(features[0]['danceability'])
            spotify_albums[album]['energy'].append(features[0]['energy'])
            spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
            spotify_albums[album]['liveness'].append(features[0]['liveness'])
            spotify_albums[album]['loudness'].append(features[0]['loudness'])
            spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
            spotify_albums[album]['tempo'].append(features[0]['tempo'])
            spotify_albums[album]['valence'].append(features[0]['valence'])
        
        #popularity is stored elsewhere
            pop = sp.track(track)
            spotify_albums[album]['popularity'].append(pop['popularity'])
            track_count+=1
#             spotify_albums[album]['artist'].append(artist['artist']) fix this; artist is stored in 'items'
        except:
            pass
        

**Scrape API for song feature per song. Set up counter to check status and estimate runtime based on number of albums**

In [139]:
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    try:
        audio_features(i)
        request_count+=1

        if request_count % 5 == 0:
            print(str(request_count) + " albums completed")
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            print('Loop #: {}'.format(request_count))
            print('Elapsed Time: {} seconds'.format(time.time() - start_time))
    except:
        pass

5 albums completed
Loop #: 5
Elapsed Time: 12.97272276878357 seconds
10 albums completed
Loop #: 10
Elapsed Time: 29.45149278640747 seconds
15 albums completed
Loop #: 15
Elapsed Time: 39.63704180717468 seconds
20 albums completed
Loop #: 20
Elapsed Time: 52.43956708908081 seconds
25 albums completed
Loop #: 25
Elapsed Time: 69.4010579586029 seconds
30 albums completed
Loop #: 30
Elapsed Time: 82.30976295471191 seconds
35 albums completed
Loop #: 35
Elapsed Time: 101.95245695114136 seconds
40 albums completed
Loop #: 40
Elapsed Time: 112.54526090621948 seconds
45 albums completed
Loop #: 45
Elapsed Time: 130.19949102401733 seconds
50 albums completed
Loop #: 50
Elapsed Time: 143.23325276374817 seconds
55 albums completed
Loop #: 55
Elapsed Time: 169.453950881958 seconds
60 albums completed
Loop #: 60
Elapsed Time: 191.38457584381104 seconds
65 albums completed
Loop #: 65
Elapsed Time: 203.44312500953674 seconds
70 albums completed
Loop #: 70
Elapsed Time: 220.5417618751526 seconds
75 a

570 albums completed
Loop #: 570
Elapsed Time: 1670.7542688846588 seconds
575 albums completed
Loop #: 575
Elapsed Time: 1680.743000984192 seconds
580 albums completed
Loop #: 580
Elapsed Time: 1695.15629196167 seconds
585 albums completed
Loop #: 585
Elapsed Time: 1708.462917804718 seconds
590 albums completed
Loop #: 590
Elapsed Time: 1722.2165696620941 seconds
595 albums completed
Loop #: 595
Elapsed Time: 1740.408302783966 seconds
600 albums completed
Loop #: 600
Elapsed Time: 1749.6333780288696 seconds
605 albums completed
Loop #: 605
Elapsed Time: 1761.5444977283478 seconds
610 albums completed
Loop #: 610
Elapsed Time: 1773.728955745697 seconds
615 albums completed
Loop #: 615
Elapsed Time: 1790.2998549938202 seconds
620 albums completed
Loop #: 620
Elapsed Time: 1810.7255868911743 seconds
625 albums completed
Loop #: 625
Elapsed Time: 1829.5904150009155 seconds
630 albums completed
Loop #: 630
Elapsed Time: 1850.4012138843536 seconds
635 albums completed
Loop #: 635
Elapsed Tim

1125 albums completed
Loop #: 1125
Elapsed Time: 3234.4843759536743 seconds
1130 albums completed
Loop #: 1130
Elapsed Time: 3248.5924887657166 seconds
1135 albums completed
Loop #: 1135
Elapsed Time: 3261.7611949443817 seconds
1140 albums completed
Loop #: 1140
Elapsed Time: 3271.490515947342 seconds
1145 albums completed
Loop #: 1145
Elapsed Time: 3283.8073117733 seconds
1150 albums completed
Loop #: 1150
Elapsed Time: 3301.009660959244 seconds
1155 albums completed
Loop #: 1155
Elapsed Time: 3320.928318977356 seconds
1160 albums completed
Loop #: 1160
Elapsed Time: 3331.276957988739 seconds
1165 albums completed
Loop #: 1165
Elapsed Time: 3342.15078496933 seconds
1170 albums completed
Loop #: 1170
Elapsed Time: 3354.1207778453827 seconds
1175 albums completed
Loop #: 1175
Elapsed Time: 3380.9287378787994 seconds
1180 albums completed
Loop #: 1180
Elapsed Time: 3392.5517637729645 seconds
1185 albums completed
Loop #: 1185
Elapsed Time: 3407.739424943924 seconds
1190 albums completed


1670 albums completed
Loop #: 1670
Elapsed Time: 4792.032841920853 seconds
1675 albums completed
Loop #: 1675
Elapsed Time: 4813.482667922974 seconds
1680 albums completed
Loop #: 1680
Elapsed Time: 4830.321569919586 seconds
1685 albums completed
Loop #: 1685
Elapsed Time: 4851.7428369522095 seconds
1690 albums completed
Loop #: 1690
Elapsed Time: 4861.408750772476 seconds
1695 albums completed
Loop #: 1695
Elapsed Time: 4879.622425794601 seconds
1700 albums completed
Loop #: 1700
Elapsed Time: 4899.349622964859 seconds
1705 albums completed
Loop #: 1705
Elapsed Time: 4912.592052936554 seconds
1710 albums completed
Loop #: 1710
Elapsed Time: 4928.633076906204 seconds
1715 albums completed
Loop #: 1715
Elapsed Time: 4940.73753786087 seconds
1720 albums completed
Loop #: 1720
Elapsed Time: 4961.427947998047 seconds
1725 albums completed
Loop #: 1725
Elapsed Time: 4988.699229955673 seconds
1730 albums completed
Loop #: 1730
Elapsed Time: 5002.083998918533 seconds
1735 albums completed
Loo

Loop #: 2215
Elapsed Time: 6379.03151679039 seconds
2220 albums completed
Loop #: 2220
Elapsed Time: 6388.502490758896 seconds
2225 albums completed
Loop #: 2225
Elapsed Time: 6400.052577972412 seconds
2230 albums completed
Loop #: 2230
Elapsed Time: 6411.602641820908 seconds
2235 albums completed
Loop #: 2235
Elapsed Time: 6440.920517921448 seconds
2240 albums completed
Loop #: 2240
Elapsed Time: 6462.782062768936 seconds
2245 albums completed
Loop #: 2245
Elapsed Time: 6477.78591299057 seconds
2250 albums completed
Loop #: 2250
Elapsed Time: 6492.218863725662 seconds
2255 albums completed
Loop #: 2255
Elapsed Time: 6501.7182948589325 seconds
2260 albums completed
Loop #: 2260
Elapsed Time: 6519.667870998383 seconds
2265 albums completed
Loop #: 2265
Elapsed Time: 6529.367702007294 seconds
2270 albums completed
Loop #: 2270
Elapsed Time: 6539.952485799789 seconds
2275 albums completed
Loop #: 2275
Elapsed Time: 6550.178877830505 seconds
2280 albums completed
Loop #: 2280
Elapsed Time:

2765 albums completed
Loop #: 2765
Elapsed Time: 7923.020042896271 seconds
2770 albums completed
Loop #: 2770
Elapsed Time: 7932.987743854523 seconds
2775 albums completed
Loop #: 2775
Elapsed Time: 7948.1648960113525 seconds
2780 albums completed
Loop #: 2780
Elapsed Time: 7962.2498388290405 seconds
2785 albums completed
Loop #: 2785
Elapsed Time: 7972.554241895676 seconds
2790 albums completed
Loop #: 2790
Elapsed Time: 7982.9277629852295 seconds
2795 albums completed
Loop #: 2795
Elapsed Time: 7999.600440979004 seconds
2800 albums completed
Loop #: 2800
Elapsed Time: 8010.654927015305 seconds
2805 albums completed
Loop #: 2805
Elapsed Time: 8029.785799741745 seconds
2810 albums completed
Loop #: 2810
Elapsed Time: 8041.591737747192 seconds
2815 albums completed
Loop #: 2815
Elapsed Time: 8055.445125818253 seconds
2820 albums completed
Loop #: 2820
Elapsed Time: 8072.531245946884 seconds
2825 albums completed
Loop #: 2825
Elapsed Time: 8099.49516582489 seconds
2830 albums completed
L

3315 albums completed
Loop #: 3315
Elapsed Time: 9430.712318897247 seconds
3320 albums completed
Loop #: 3320
Elapsed Time: 9440.139367818832 seconds
3325 albums completed
Loop #: 3325
Elapsed Time: 9450.756073951721 seconds
3330 albums completed
Loop #: 3330
Elapsed Time: 9462.322823762894 seconds
3335 albums completed
Loop #: 3335
Elapsed Time: 9479.06882095337 seconds
3340 albums completed
Loop #: 3340
Elapsed Time: 9491.502033948898 seconds
3345 albums completed
Loop #: 3345
Elapsed Time: 9508.295868873596 seconds
3350 albums completed
Loop #: 3350
Elapsed Time: 9520.085488796234 seconds
3355 albums completed
Loop #: 3355
Elapsed Time: 9532.121498823166 seconds
3360 albums completed
Loop #: 3360
Elapsed Time: 9541.552551746368 seconds
3365 albums completed
Loop #: 3365
Elapsed Time: 9550.12312078476 seconds
3370 albums completed
Loop #: 3370
Elapsed Time: 9560.623300790787 seconds
3375 albums completed
Loop #: 3375
Elapsed Time: 9569.055084943771 seconds
3380 albums completed
Loop 

3860 albums completed
Loop #: 3860
Elapsed Time: 10991.18303990364 seconds
3865 albums completed
Loop #: 3865
Elapsed Time: 11001.788510799408 seconds
3870 albums completed
Loop #: 3870
Elapsed Time: 11014.93935084343 seconds
3875 albums completed
Loop #: 3875
Elapsed Time: 11022.70458483696 seconds
3880 albums completed
Loop #: 3880
Elapsed Time: 11041.443923950195 seconds
3885 albums completed
Loop #: 3885
Elapsed Time: 11051.325444698334 seconds
3890 albums completed
Loop #: 3890
Elapsed Time: 11063.68151807785 seconds
3895 albums completed
Loop #: 3895
Elapsed Time: 11075.12345290184 seconds
3900 albums completed
Loop #: 3900
Elapsed Time: 11089.76656293869 seconds
3905 albums completed
Loop #: 3905
Elapsed Time: 11103.260427951813 seconds
3910 albums completed
Loop #: 3910
Elapsed Time: 11119.516684055328 seconds
3915 albums completed
Loop #: 3915
Elapsed Time: 11142.586165904999 seconds
3920 albums completed
Loop #: 3920
Elapsed Time: 11154.317501783371 seconds
3925 albums comple

Loop #: 4400
Elapsed Time: 12639.514437913895 seconds
4405 albums completed
Loop #: 4405
Elapsed Time: 12649.519199848175 seconds
4410 albums completed
Loop #: 4410
Elapsed Time: 12659.997193813324 seconds
4415 albums completed
Loop #: 4415
Elapsed Time: 12673.592970848083 seconds
4420 albums completed
Loop #: 4420
Elapsed Time: 12682.591278076172 seconds
4425 albums completed
Loop #: 4425
Elapsed Time: 12691.152079105377 seconds
4430 albums completed
Loop #: 4430
Elapsed Time: 12702.046243906021 seconds
4435 albums completed
Loop #: 4435
Elapsed Time: 12712.72456908226 seconds
4440 albums completed
Loop #: 4440
Elapsed Time: 12721.552104949951 seconds
4445 albums completed
Loop #: 4445
Elapsed Time: 12734.100569725037 seconds
4450 albums completed
Loop #: 4450
Elapsed Time: 12761.65212893486 seconds
4455 albums completed
Loop #: 4455
Elapsed Time: 12781.692260026932 seconds
4460 albums completed
Loop #: 4460
Elapsed Time: 12791.164241790771 seconds
4465 albums completed
Loop #: 4465
E

**Create dictionary of song features per track to translate into dataframe**

In [145]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []

for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

67605

In [146]:
len(dic_df['popularity'])

67560

In [147]:
#arrays are not the same length, just going to drop nulls
df = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in dic_df.items()]))
df.head()

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,1,3uwYgNxFDfx1GoLB6tLoUn,Good Day,spotify:track:3uwYgNxFDfx1GoLB6tLoUn,0.110000,0.684,0.677,0.009750,0.3400,-7.355,0.0401,142.048,0.370,73.0
1,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,2,732E6ibFkQR5lNIPEeiEnx,Choker,spotify:track:732E6ibFkQR5lNIPEeiEnx,0.306000,0.700,0.664,0.000019,0.2490,-8.211,0.0394,142.009,0.585,76.0
2,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,3,2nG54Y4a3sH9YpfxMolOyi,Shy Away,spotify:track:2nG54Y4a3sH9YpfxMolOyi,0.000818,0.588,0.856,0.073900,0.2380,-5.242,0.0316,96.966,0.635,80.0
3,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,4,0yzj6eBs5a6X6d3P5qaQ5J,The Outside,spotify:track:0yzj6eBs5a6X6d3P5qaQ5J,0.050500,0.785,0.623,0.125000,0.0539,-6.923,0.0403,100.991,0.711,74.0
4,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,5,60dwJ8dnqZ4WrIdiu7LkNR,Saturday,spotify:track:60dwJ8dnqZ4WrIdiu7LkNR,0.003960,0.687,0.890,0.000939,0.0436,-5.990,0.0344,113.060,0.854,78.0


In [148]:
#can drop track number and id (redundant)
df.drop(columns=['id'], inplace=True)

In [149]:
#drop any nulls
df.dropna(inplace=True)

In [150]:
#export to csv
df.to_csv('data/alternative_finalv3.csv', index=False) #start here

In [151]:
#read back in to double check for nulls
df = pd.read_csv('data/alternative_finalv3.csv')

In [152]:
df.isnull().sum()

album               0
track_number        0
name                0
uri                 0
acousticness        0
danceability        0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
dtype: int64

In [153]:
df.head()

,album,track_number,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,1,Good Day,spotify:track:3uwYgNxFDfx1GoLB6tLoUn,0.110000,0.684,0.677,0.009750,0.3400,-7.355,0.0401,142.048,0.370,73.0
1,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,2,Choker,spotify:track:732E6ibFkQR5lNIPEeiEnx,0.306000,0.700,0.664,0.000019,0.2490,-8.211,0.0394,142.009,0.585,76.0
2,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,3,Shy Away,spotify:track:2nG54Y4a3sH9YpfxMolOyi,0.000818,0.588,0.856,0.073900,0.2380,-5.242,0.0316,96.966,0.635,80.0
3,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,4,The Outside,spotify:track:0yzj6eBs5a6X6d3P5qaQ5J,0.050500,0.785,0.623,0.125000,0.0539,-6.923,0.0403,100.991,0.711,74.0
4,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,5,Saturday,spotify:track:60dwJ8dnqZ4WrIdiu7LkNR,0.003960,0.687,0.890,0.000939,0.0436,-5.990,0.0344,113.060,0.854,78.0


In [154]:
df.shape

(67560, 14)

**Post capstone:**

- fix artist scrape issue (wasn't needed for this but I will need eventually)
- scrape additional genres